## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων.

Συμπληρώστε το όνομα (NAME) και το AEM σας παρακάτω:

In [1]:
NAME = "Konstantinos Kalamaras"
AEM = "9716"

**1** Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων breast cancer, χωρίστε το σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70%/30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0). Το σύνολο αφορά τη διάγνωση καρκίνου του μαστού με βάση μεταβλητές που υπολογίζονται από μια ψηφιοποιημένη εικόνα δείγματος μάζας μαστού που λήφθηκε μέσω αναρρόφησης λεπτής βελόνας (FNA). (2 μονάδες)

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Load the breast cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Split the data into training and testing sets with a 70%/30% ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [3]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.1779
assert round(X_test[0][8], 5) == 0.2116

**2** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. (4 μονάδες)

In [4]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        self.estimator = estimator
        self.models = []

    def fit(self, X_train, y_train):
        self.models = []
        n_features = X_train.shape[1]

        for i in range(n_features):
            # Create a new model by cloning the estimator
            model = clone(self.estimator)
            
            # Select all columns except the i-th column
            X_train_subspace = np.delete(X_train, i, axis=1)
            
            # Fit the model on the subspace
            model.fit(X_train_subspace, y_train)
            
            # Append the model to the models list
            self.models.append(model)

    def predict(self, X):
        n_features = X.shape[1]
        predictions = np.zeros((X.shape[0], n_features))

        for i, model in enumerate(self.models):
            # Select all columns except the i-th column
            X_subspace = np.delete(X, i, axis=1)
            
            # Predict with the model
            predictions[:, i] = model.predict(X_subspace)

        # Majority vote for final prediction
        y_pred = np.round(np.mean(predictions, axis=1)).astype(int)
        return y_pred

In [5]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.9006

**3** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. (4 μονάδες)

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        self.n_estimators = n_estimators
        self.estimator = estimator
        self.estimators_ = []
        self.estimator_weights_ = []
        
    def fit(self, X_train, y_train):
        m = len(X_train)
        # Initialize sample weights
        weights = np.ones(m) / m
        
        # Loop through the specified number of estimators
        for _ in range(self.n_estimators):
            estimator = clone(self.estimator)  # Clone the base estimator
            estimator.fit(X_train, y_train, sample_weight=weights)  # Fit the estimator with current weights
            y_pred = estimator.predict(X_train)  # Predict on training data
            
            # Calculate weighted error
            weighted_error = np.sum(weights * (y_train != y_pred))
            estimator_weight = np.log((1 - weighted_error) / weighted_error)  # Calculate estimator weight
            
            # Check if error is too high or too low
            if weighted_error >= 0.5 or weighted_error == 0:
                break
            
            # Update weights
            updated_weights = np.exp(estimator_weight * (y_train != y_pred))
            weights *= updated_weights
            weights /= np.sum(weights)  # Normalize weights
        
            # Store the fitted estimator and its weight
            self.estimators_.append(estimator)
            self.estimator_weights_.append(estimator_weight)


    def predict(self, X):
        class_votes = np.zeros((len(X), len(np.unique(y_train)))) # Initialize array to store class votes

        # Loop through fitted estimators and their weights
        for estimator, estimator_weight in zip(self.estimators_, self.estimator_weights_):
            y_pred = estimator.predict(X)  # Predict class labels
            # Accumulate votes weighted by estimator weight
            for i in range(len(X)):
                class_votes[i, y_pred[i]] += estimator_weight
        
        return np.argmax(class_votes, axis=1)  # Return class labels with maximum votes

In [7]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591


In [8]:
# Ίδιο αποτέλεσμα και με τη κλάση της sklearn
ab = AdaBoostClassifier(n_estimators=20, algorithm="SAMME", estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591